In [7]:
import os
from pathlib import Path
from dotenv import load_dotenv

import json
import datetime

from openai import AzureOpenAI
from pydantic import BaseModel, Field

from prompts.StateDiagram import SYSTEM_PROMPT, user_message, example_message
from utils import save_solution_puml, save_chat_json, save_chat_text

In [8]:
SEED = 1234

base_path = Path().cwd()

cpp_path = base_path / "dishwasher_cpp/host"

chat_path = base_path / "chats" / "general_examples"
chat_path.mkdir(parents=True, exist_ok=True)


few_shots_path = base_path / "diagrams" / "state_diagram_general_examples" / "ground_truth"
state_diagram_path = base_path / "diagrams" / "state_diagram_general_examples" / "generated"

core_components = json.load((base_path / "core_components_openai.json").open("r"))

load_dotenv(override=True)

True

In [9]:
# collect only cpp and puml files for all available general examples
example_files = {}

for f in few_shots_path.iterdir():

    cpp_files = list(f.glob("*.cpp"))
    image_files = list(f.glob("*.jpg"))

    if len(cpp_files) > 0 and len(image_files) > 0:
        example_files[f.name] = [cpp_files, image_files[0]]

In [10]:
# format chatgpt response format
class Solution(BaseModel):
    solution: str = Field(description="PlantUML solution that start with '@startuml' and end with '@enduml'")

In [11]:
# generate simulated conversation for each example
examples = [example_message(c, p) for c, p in example_files.values()]
# flatten
examples = [e for es in examples for e in es]

In [12]:
for target_component, code_files_raw in core_components.items():

    # get code files for the target component
    code_files = []
    for c in code_files_raw:
        code_files.append(base_path / c.replace("\\", "/").replace(".h", ".cpp"))

    if len(code_files) == 0:
        print(f"No code files found for component: {target_component}")
        continue

    target_component = target_component.lower()
    print(f"Processing component: {target_component}")

    # User Prompt containing target cpp
    user_prompt = user_message(code_files)
    # System Prompt
    messages = SYSTEM_PROMPT + examples + user_prompt

    client = AzureOpenAI(
        api_key=os.getenv("AZURE_API_KEY"),
        api_version=os.getenv("AZURE_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_API_BASE"),
    )

    # call ChatGPT completion API
    response = client.beta.chat.completions.parse(
        model=os.getenv("MODEL"),
        temperature=0.2,
        messages=messages,
        seed=SEED,
        response_format=Solution,
    )

    # Add response to messages
    messages.append(
        {"role": "assistant", "content": response.choices[0].message.content}
    )
    # parse output
    solution = json.loads(response.choices[0].message.content)["solution"]

    # save solution to puml file, render it, and save messages
    chat_path_component = chat_path / target_component
    chat_path_component.mkdir(parents=True, exist_ok=True)

    chat_filename = f"state_general_{target_component}_{response.model}"
    date_postfix = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

    save_solution_puml(
        solution, state_diagram_path, target_component, response.model, date_postfix
    )
    save_chat_json(
        messages, solution, response, chat_path_component, chat_filename, date_postfix
    )
    save_chat_text(messages, chat_path_component, chat_filename, date_postfix)

Processing component: abstractfactory
Processing component: dishwasher
Processing component: heater
Processing component: jet
Processing component: tank
